### Ajuste da rede MLP - com tunning de hiperparâmetros utilizando otimização bayesiana

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, matthews_corrcoef
from sklearn.model_selection import cross_val_score
from sklearn.base import BaseEstimator, ClassifierMixin

In [2]:
from skopt import gp_minimize, BayesSearchCV
from skopt.space import Real, Integer,Categorical
from skopt.utils import use_named_args

In [3]:
from scipy.stats import ks_2samp

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
base_treino = pd.read_csv('base_treino.csv')

In [6]:
X_treino = base_treino.iloc[:,:-1]
y_treino = base_treino['MAU']

In [7]:
#Espaço de busca dos hiperparametros
param_space = {
    'activation': ['relu', 'logistic','tanh'],
    'solver': ['adam', 'lbfgs'],
    'alpha': (1e-4, 1e-1, 'log-uniform'),
    'learning_rate': ['constant', 'adaptive'],
}

A função BayesSearchCV não aceita como parâmetro as tuplas que descrevem as camadas de neurônios da rede MLP no pacote scikit-learn. Dessa forma, uma alternativa é criar um laço variando os neurônios e deixar o otimizador bayesiano trabalhar em cada configuração otimizando os restante dos parâmetros, como abaixo:

In [8]:
%%time
neuronios = [8,10,20,(8,8),(10,10),(20,20),(8,8,8),(10,10,10),(20,20,20)]
for i in neuronios:
    mlp = MLPClassifier(max_iter=200 , hidden_layer_sizes=i, random_state=42)
    
    # Realizar a busca bayesiana dos melhores parâmetros
    search = BayesSearchCV(mlp, param_space, n_iter=20, cv=5, random_state=42,scoring='roc_auc')
    search.fit(X_treino, y_treino)
    
    print(f'Qtd neurônios: {i}')
    print(f'Melhor estimador:{search.best_estimator_}')
    print(f'Melhor AUC:{search.best_score_:.4f}\n')


Qtd neurônios: 8
Melhor estimador:MLPClassifier(activation='tanh', alpha=0.09838436736084676,
              hidden_layer_sizes=8, learning_rate='adaptive', random_state=42)
Melhor AUC:0.6756

Qtd neurônios: 10
Melhor estimador:MLPClassifier(activation='tanh', alpha=0.09916570116754046,
              hidden_layer_sizes=10, learning_rate='adaptive', random_state=42)
Melhor AUC:0.6758

Qtd neurônios: 20
Melhor estimador:MLPClassifier(activation='logistic', alpha=0.1, hidden_layer_sizes=20,
              learning_rate='adaptive', random_state=42)
Melhor AUC:0.6765

Qtd neurônios: (8, 8)
Melhor estimador:MLPClassifier(activation='logistic', hidden_layer_sizes=(8, 8),
              learning_rate='adaptive', random_state=42)
Melhor AUC:0.6762

Qtd neurônios: (10, 10)
Melhor estimador:MLPClassifier(activation='logistic', alpha=0.0022044857650599537,
              hidden_layer_sizes=(10, 10), learning_rate='adaptive',
              random_state=42)
Melhor AUC:0.6766

Qtd neurônios: (20, 20)
Mel

Como a melhor simulação de hiperparâmetros foi com 2 camadas de 20 neurônios, vamos fazer a busca de novo com essa configuração deixando o otimizador poder realizar mais iterações pra ver se melhoramos as metricas de avaliação.

In [9]:
mlp = MLPClassifier(max_iter=200,hidden_layer_sizes=(20,20), random_state=42)

# Realizar a busca bayesiana dos melhores parâmetros
search = BayesSearchCV(mlp, param_space, n_iter=25, cv=5, random_state=42,scoring='roc_auc')
search.fit(X_treino, y_treino)

print(f'Qtd neurônios: {i}')
print(f'Melhor estimador:{search.best_estimator_}')
print(f'Melhor AUC:{search.best_score_:.4f}\n')

Qtd neurônios: (20, 20, 20)
Melhor estimador:MLPClassifier(activation='logistic', alpha=0.028411231508773168,
              hidden_layer_sizes=(20, 20), random_state=42)
Melhor AUC:0.6768



In [10]:
MLP_best = search.best_estimator_

Lendo o arquivo de teste para cálculo das métricas de avaliação

In [11]:
base_teste = pd.read_csv('base_teste.csv')

In [12]:
X_teste = base_teste.iloc[:,:-1]
y_teste = base_teste['MAU']

In [13]:
y_preds = MLP_best.predict_proba(X_teste)[:,1]

In [14]:
roc_auc_score(y_teste , y_preds)

0.700668225094945

In [15]:
y_preds_bin = pd.Series(y_preds).apply(lambda x: 1 if x> 0.34 else 0)

In [16]:
matthews_corrcoef( base_teste['MAU'],  y_preds_bin )

0.2834470398388893

In [17]:
ks_2samp(y_preds[y_teste ==0],y_preds[y_teste ==1])

KstestResult(statistic=0.2951764088354318, pvalue=1.1530218413537098e-35, statistic_location=0.269688335481985, statistic_sign=1)